In [ ]:
%matplotlib

In [ ]:
%matplotlib inline

In [ ]:
pd.set_option('display.max_rows',    100)      # DataFrame 데이터 확인 시 최대 표시 행 수
pd.set_option('display.max_columns', 100)      # DataFrame 데이터 확인 시 최대 표시 열 수

## 라이브러리 import 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import tensorflow as tf
from tensorflow import keras

## 데이터 불러오기

In [ ]:
Fold       = 5
Fs         = 12800  # Sampling Frequency
NoOfSensor = 1


# k-fold 학습/검증 데이터
for i in range(Fold):
    
    path1 = './K_FoldData/CNN_Training_Fold%d'%(i+1)
    path2 = './K_FoldData/CNN_Validation_Fold%d'%(i+1)
    c1 = 'temp_Training_Fold%d   = np.array(pd.read_csv(path1, sep=",", header=None))'%(i+1)
    c2 = 'temp_Validation_Fold%d = np.array(pd.read_csv(path2, sep=",", header=None))'%(i+1)
    exec(c1)
    exec(c2)

# K-fold 학습/검증 레이블
TrainingFold_Label   = np.array(pd.read_csv('./K_FoldData/TrainingFold_Label_forANN'  , sep=",", header=None))
ValidationFold_Label = np.array(pd.read_csv('./K_FoldData/ValidationFold_Label_forANN', sep=",", header=None))
    
    
# 전체 학습용 데이터
temp_Training_All  = np.array(pd.read_csv('./K_FoldData/CNN_Training_All', sep = ",", header = None))
Training_All_Label = np.array(pd.read_csv('./K_FoldData/Training_All_Label_forANN', sep = ",", header = None))

print(temp_Training_Fold1.shape)
print(temp_Validation_Fold1.shape)
print(TrainingFold_Label.shape)
print(ValidationFold_Label.shape)
print(temp_Training_All.shape)

## STFT 실행 (Spectrogram 생성)

In [ ]:
# STFT test

f,t,Sxx = signal.spectrogram(temp_Training_All, Fs, nperseg=78)
print("STFT Result            :", Sxx.shape)

# Data Reshape for CNN
Training_All = Sxx.reshape(Sxx.shape[0], Sxx.shape[1], Sxx.shape[2], NoOfSensor)
print("Reshape_for_CNN Result :", Training_All.shape)

In [ ]:
for i in range(Fold):
    s1 = 'f,t,Sxx1 = signal.spectrogram(temp_Training_Fold%d, Fs, nperseg=78)'   %(i+1)
    s2 = 'f,t,Sxx2 = signal.spectrogram(temp_Validation_Fold%d, Fs, nperseg=78)' %(i+1)
    exec(s1)
    exec(s2)
    
    s3 = 'Training_Fold%d   = Sxx1.reshape(Sxx1.shape[0], Sxx1.shape[1], Sxx1.shape[2], NoOfSensor)' %(i+1)
    s4 = 'Validation_Fold%d = Sxx2.reshape(Sxx2.shape[0], Sxx2.shape[1], Sxx2.shape[2], NoOfSensor)' %(i+1)
    exec(s3)
    exec(s4)


print(f.shape, t.shape, Training_Fold1.shape, Validation_Fold1.shape)

## CNN(Convolution Neural Network) hyperparameter 설정

In [ ]:
learningRate  = 0.001
iteration     = 500

## Keras 기반 CNN 구조(Architecture) 설계

In [ ]:
def CNN_model(input_data):
    model = keras.Sequential()
    model.add(keras.layers.Conv2D(4, kernel_size=(3,3), strides=(1, 1), padding='same', activation='relu',
                                  input_shape=(input_data.shape[1],input_data.shape[2],input_data.shape[3]))) # Convolution layer 1
    model.add(keras.layers.MaxPooling2D(pool_size = (2,2), strides=(2,2)))                                    # Pooling layer 1
    model.add(keras.layers.Conv2D(8, kernel_size=(3,3), strides=(1, 1), padding='same', activation='relu'))   # Convolution layer 2
    model.add(keras.layers.GlobalAveragePooling2D())                                                          # Global Average Pooling
    model.add(keras.layers.Dense(units = 2, activation='softmax'))                                            # Output Layer

    
    model.compile(optimizer= keras.optimizers.Adam(learning_rate = learningRate),
                  loss=keras.losses.categorical_crossentropy,
                  metrics=['accuracy'])
    return model

## CNN(Convolution Neural Network) 학습 및 평가 방법

In [ ]:
model = CNN_model(Training_Fold1)
model.summary()

In [ ]:
tf.random.set_seed(777)

Data      = Training_Fold1
Data_Val  = Validation_Fold1
Label     = TrainingFold_Label
Label_Val = ValidationFold_Label

model = CNN_model(Data)

hist = model.fit(Data, Label, epochs=iteration, verbose = 0)
Loss, Accuracy = model.evaluate(Data_Val,  Label_Val, verbose=0)

Accuracy

## CNN(Convolution Neural Network) 학습 및 검증

In [ ]:
Label        = TrainingFold_Label
Label_Val    = ValidationFold_Label
Accuracy_sum = 0

print('[Result of K-fold Cross Validation] \n')

for i in range(Fold):
    tf.random.set_seed(777)
    
    s1= 'Data = Training_Fold%d'%(i+1)
    s2= 'Data_Val = Validation_Fold%d'%(i+1)
    exec(s1)
    exec(s2)
    
    model = CNN_model(Data)
    
# train my model
    hist = model.fit(Data, Label, epochs=iteration, verbose = 0)
    Loss, Accuracy = model.evaluate(Data_Val,  Label_Val, verbose=0)
    Accuracy_sum = Accuracy_sum + Accuracy
    print('Fold {}: {:.2f}%'.format((i+1), Accuracy*100))

print('* Average accuracy : {:.2f}%'.format((Accuracy_sum/Fold)*100))

## 전체 학습 데이터로 NN 학습 및 모델 저장

In [ ]:
PrintAccPerEpochs = 200

class AccuracyPerEpoch(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        keras.callbacks.Callback()
        if epoch%PrintAccPerEpochs == 0:
            print("{} Epochs Accuracy : {:.2f}% ".format(epoch, logs["accuracy"]*100))

In [ ]:
tf.random.set_seed(777)    

Data  = Training_All
Label = Training_All_Label

model = CNN_model(Training_All)
hist = model.fit(Data, Label, epochs=iteration, verbose = 0, callbacks = [AccuracyPerEpoch()])

print('Final Accuracy      : {:.2f}%'.format(hist.history['accuracy'][-1]*100))

In [ ]:
# 학습 과정 확인

fig, loss_ax = plt.subplots()
acc_ax = loss_ax.twinx()

loss_ax.plot(hist.history['loss'], 'y', label='train loss', c = 'tab:red')
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
loss_ax.legend(loc='upper left')

acc_ax.plot(hist.history['accuracy'], 'b', label='train acc', c = 'tab:blue')
acc_ax.set_ylabel('accuracy')
acc_ax.legend(loc='lower left')

plt.show()

In [ ]:
# 모델 저장
model.save("./MLmodels/Simple_CNN_model.h5")

## NN model 불러와서 진단

In [ ]:
# 모델 불러오기
LoadedModel = keras.models.load_model("./MLmodels/Simple_CNN_model.h5")

In [ ]:
temp_Test_Data         = np.array(pd.read_csv('./K_FoldData/CNN_Test_Data' , sep=",", header=None))
Test_Label_forANN = np.array(pd.read_csv('./K_FoldData/Test_Label_forANN', sep=",", header=None))

In [ ]:
f,t,Test_Data = signal.spectrogram(temp_Test_Data, Fs, nperseg = 78)
print("STFT Result            :", Training_All.shape)

# Data Reshape for CNN
Test_Data = Test_Data.reshape(Test_Data.shape[0], Test_Data.shape[1], Test_Data.shape[2], NoOfSensor)
print("Reshape_for_CNN Result :", Test_Data.shape)

In [ ]:
Loss, Accuracy = LoadedModel.evaluate(Test_Data,  Test_Label_forANN, verbose=0)
print('[Performance of CNN model] \n')
print('Accuracy : {:.2f}%'.format(Accuracy*100))

In [ ]:
# 예측값 확인
Predict_y = LoadedModel.predict(Test_Data)
pd.DataFrame(Predict_y)